# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [4]:
# Start by installing the dependencies
%pip install python-frontmatter getorg --upgrade
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\jimeg\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")

In [6]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

In [ ]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data:
        continue

    # Prepare the description
    #title = data['title'].strip()
    #venue = data['venue'].strip()
    #location = data['location'].strip()
    #description = f"{title}<br />{venue}; {location}"

    title = data.get('title', '').strip() if data.get('title') else ''
    venue = data.get('venue', '').strip() if data.get('venue') else ''
    #location = data.get('location', '').strip() if data.get('location') else '' 
    location = (data.get('location') or "Unknown").strip() if data.get('location') else ''

    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

Double-dipping in Environmental Markets under Uncoordinated Regulators or under a Regulator who Ignores Pollutants’ Complementarities<br />Missouri Valley Economic Association Conference; St Louis, MO Saint Louis, Missouri, United States
Double-dipping in Environmental Markets under Uncoordinated Regulators or under a Regulator who Ignores Pollutants’ Complementarities<br />Heartland Environmental and Resource Economics Workshop at Illinois; Champaign, IL Champaign, Champaign County, Illinois, United States
Double-dipping in Environmental Markets under Two Second Best Scenario<br />European Association of Environmental and Resource Economists Annual Conference; Helsinki, Finland Helsinki, Helsingin seutukunta, Uusimaa, Manner-Suomi, Suomi / Finland
Adoption and Attitudes towards Cover Crops in Iowa<br />Agricultural and Applied Economics Association Annual Meeting; San Francisco, CA San Francisco, California, United States
Adoption and Attitudes towards Cover Crops in Iowa<br />Missour

AttributeError: 'NoneType' object has no attribute 'strip'

In [ ]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)